In [1]:
from theano import *
from lasagne.layers import EmbeddingLayer, InputLayer, get_output
import lasagne
import lasagne.layers
import theano.tensor as T
import theano
import numpy as np

Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
l_in = lasagne.layers.InputLayer((10,50))

In [3]:
l_output = lasagne.layers.DenseLayer(l_in, num_units=2, name='output')

/home/matthew/.virtualenvs/nlp-convnet/lib/python2.7/site-packages/lasagne/init.py:86: UserWarning: The uniform initializer no longer uses Glorot et al.'s approach to determine the bounds, but defaults to the range (-0.01, 0.01) instead. Please use the new GlorotUniform initializer to get the old behavior. GlorotUniform is now the default for all layers.
  warnings.warn("The uniform initializer no longer uses Glorot et al.'s "


In [4]:
out = lasagne.layers.get_output(l_output)

/home/matthew/.virtualenvs/nlp-convnet/lib/python2.7/site-packages/lasagne/layers/helper.py:69: UserWarning: get_all_layers() has been changed to return layers in topological order. The former implementation is still available as get_all_layers_old(), but will be removed before the first release of Lasagne. To ignore this warning, use `warnings.filterwarnings('ignore', '.*topo.*')`.
  warnings.warn("get_all_layers() has been changed to return layers in "


In [5]:
print out

Elemwise{mul,no_inplace}.0


In [6]:
func = theano.function([l_in.input_var], out)

In [7]:
target_v = (np.random.randint(0,2,10)).astype('int32')
target_t = np.random.randn(10, 50)

In [8]:
target_v.shape, target_t.shape

((10,), (10, 50))

In [9]:
func(target_t)

array([[ 3.61084658,  0.46393442],
       [ 1.45270656,  0.        ],
       [ 0.        ,  1.72213931],
       [ 0.        ,  1.79623763],
       [ 0.33651443,  1.30487711],
       [ 0.        ,  0.17773661],
       [ 0.        ,  0.        ],
       [ 0.34447744,  0.53940711],
       [ 0.13516176,  1.30733922],
       [ 0.        ,  0.        ]])

In [10]:
all_params = lasagne.layers.get_all_params(l_output)

In [11]:
theano.function([l_in.input_var], (out, T.argmax(out, axis=1)))(target_t)[1]

array([0, 0, 1, 1, 1, 1, 0, 1, 1, 0])

In [12]:
y_value = T.ivector('y')
out_values = T.argmax(out, axis=1)
#loss = (y_value - out_values) ** 2
#loss_old = (y_value - ) ** 2
loss = theano.tensor.nnet.binary_crossentropy(out_values + 1, y_value + 1)

There was some issue with the shape of the ouput vector being [10,1] and the shape of the Y values being [10]

this hack above is computing the loss by transforming the output vector into a single dimentional item, and then we compute the difference between these two terms and square it

maybe simply taking the transpose of the y_value matrix can make these two values compatiable

In [13]:
update = lasagne.updates.nesterov_momentum(loss.mean(), all_params, 0.1)

In [14]:
train = theano.function([l_in.input_var, y_value], (out_values, y_value, loss, loss.mean(), update.values()[0]), updates=update)

In [15]:
update

OrderedDict([(output.W, Elemwise{add,no_inplace}.0), (output.b, Elemwise{add,no_inplace}.0), (<TensorType(float64, matrix)>, Elemwise{sub,no_inplace}.0), (<TensorType(float64, vector)>, Elemwise{sub,no_inplace}.0)])

In [16]:
for i in xrange(1000):
    _,_,_,loss_v,_= train(target_t, target_v)
print loss_v

nan


In [30]:
train(target_t, target_v)

[array([0, 0, 1, 1, 1, 1, 0, 1, 1, 0]),
 array([0, 1, 0, 1, 0, 1, 1, 1, 0, 1], dtype=int32),
 array([ nan, -inf,  nan,  nan,  nan,  nan, -inf,  nan,  nan, -inf]),
 array(nan),
 array([[ 0.04335611,  0.05118995],
        [-0.01719553, -0.118362  ],
        [-0.20536817,  0.07178673],
        [-0.07122489,  0.2425208 ],
        [-0.29429854, -0.04354277],
        [ 0.17944093,  0.08944575],
        [ 0.30838578,  0.04708556],
        [ 0.0038523 , -0.22174831],
        [-0.13206892, -0.03355977],
        [ 0.23500965, -0.06633459],
        [-0.27021436, -0.07395862],
        [ 0.23376679,  0.24184481],
        [ 0.09989518,  0.29724972],
        [ 0.32914167, -0.12673656],
        [-0.32831812,  0.17769959],
        [ 0.14105618,  0.25718294],
        [ 0.1966741 , -0.26075592],
        [-0.14865006,  0.19150122],
        [ 0.31101563,  0.17198993],
        [-0.18876439, -0.08521521],
        [ 0.11649479,  0.27220297],
        [-0.15419455, -0.23373412],
        [ 0.00988218,  0.0453865

In [18]:
func(target_t), target_v

(array([[ 3.61084658,  0.46393442],
        [ 1.45270656,  0.        ],
        [ 0.        ,  1.72213931],
        [ 0.        ,  1.79623763],
        [ 0.33651443,  1.30487711],
        [ 0.        ,  0.17773661],
        [ 0.        ,  0.        ],
        [ 0.34447744,  0.53940711],
        [ 0.13516176,  1.30733922],
        [ 0.        ,  0.        ]]),
 array([0, 1, 0, 1, 0, 1, 1, 1, 0, 1], dtype=int32))